<a href="https://colab.research.google.com/github/Luky-ALT/Projects_by_Python/blob/main/CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CSV

In [ ]:
import csv
from typing import List

import requests
from bs4 import BeautifulSoup

URL = "http://heroes3.cz/hraci"


def hlavni() -> None:
    odpoved = ziskej_odpoved()
    naparsovano = vytahni_udaje(odpoved)

    tabulka = hledej_tabulku(naparsovano)
    radky = hledej_radky(tabulka)

    konecne_udaje = (hraci_info(row) for row in radky)
    uloz_csv(list(konecne_udaje))


def ziskej_odpoved():
    with requests.Session() as s:
        return s.get(URL)


def vytahni_udaje(resp):
    return BeautifulSoup(resp.text, "html.parser")


def hledej_tabulku(cont):
    return cont.find("table", {"class": "tab_top"})


def hledej_radky(tabl) -> list:
    return tabl.find_all("tr")[1:]


def hraci_info(h2) -> dict:
    try:
        poradi = h2.find_all("td")[0].text
        hrac = h2.find_all("td")[2].text
        body = h2.find_all("td")[3].text
        uspesnost = h2.find_all("td")[7].text
        return {"poradi": poradi, "hrac": hrac, "body": body, "uspesnost": uspesnost}

    except AttributeError:
        print("Indexy u jednotlivych bunek v radku nejsou v poradku")


def uloz_csv(data: List[dict]):
    with open("hraci.csv", "a", newline="") as csv_soubor:
        zahlavi = ["PORADI", "UZIVATEL", "BODY", "USPESNOST"]
        writer = csv.DictWriter(csv_soubor, fieldnames=zahlavi)
        writer.writeheader()

        for index, _ in enumerate(data):
            writer.writerow(
                {
                    "PORADI": data[index]["poradi"],
                    "UZIVATEL": data[index]["hrac"],
                    "BODY": data[index]["body"],
                    "USPESNOST": data[index]["uspesnost"],
                }
            ) 


if __name__ == "__main__":
    hlavni()

In [ ]:
import csv

import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.ebay.co.uk/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=canon+m50&_sacat=0"




def get_data(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    return soup


def parse(soup):
    productslist = []
    results = soup.find_all("div", {"class" : "s-item__info clearfix"})
    for item in results:
        products = {
            'title': item.find('div', {'class': 's-item__title'}).text,
            'soldprice': item.find('span', {'class': 's-item__price'}),
            'bids': item.find("span", {'class': 's-item__bids'}),
            'link': item.find("a", {"class":"s-item__link"}),
        }
        productslist.append(products)
    return productslist

def output(productslist):
    productsdf = pd.DataFrame(productslist)
    productsdf.to_csv("output.csv", index=False)
    print("saved to csv")
    return

soup = get_data(URL)
productslist = parse(soup)
output(productslist)